In [282]:
from typing import List
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,qiskit
from qiskit import Aer, IBMQ, transpile, execute, assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.providers.ibmq import least_busy
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram
from qiskit.algorithms import AmplificationProblem
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover

def qram(data: List) -> QuantumCircuit:
    """A naive implementation of QRAM.
    Given the data list, it prepares the state \sum_x |x>|D(x)>,
    where the |x> is the address (index) and |D(x)> is the data in the corresponding address,
    e,g, if the list is [3, 5, 1], then the state is (|0>|3> + |1>|5> + |2>|1>) / sqrt(3).
    Note: this function will note construct the superposition of address register,
    remember to apply H gates by yourself.
    Args:
        data: The Python List object of the data you want to search.
    Returns:
        circuit: QuantumCircuit for QRAM.
    """
    num_addr_bits = int(np.ceil(np.log2(len(data))))
    num_data_bits = int(np.ceil(np.log2(max(data))))
    address_reg = QuantumRegister(num_addr_bits)
    data_reg = QuantumRegister(num_data_bits)
    circuit = QuantumCircuit(address_reg, data_reg, name='QRAM')
    
    for id_addr in range(len(data)):
        # load address index
        id_str = format(id_addr, 'b').zfill(len(address_reg))
        for i_id_str, b_id_str in enumerate(id_str):
            if b_id_str == '0': circuit.x(address_reg[i_id_str])
        # load data
        data_str = format(data[id_addr], 'b').zfill(len(data_reg))
        for i_data_str, b_data_str in enumerate(data_str):
            if b_data_str == '1': circuit.mct(address_reg, data_reg[i_data_str])

        for i_id_str, b_id_str in enumerate(id_str):
            if b_id_str == '0': circuit.x(address_reg[i_id_str])
        circuit.barrier()
    return circuit


def oracle_c(num_data: int) -> QuantumCircuit:
    """Oracle
    This oracle is used for determining if the data == 5, not the index.
    Args:
        num_data: Number of qubits in data register.
    Returns:
        Oracle circuit.
    """
    data_reg = QuantumRegister(num_data)
    ancilla = QuantumRegister(1)
    circuit = QuantumCircuit(data_reg, ancilla, name='Oracle')
    
    circuit.x(data_reg[1])
    circuit.mcx(data_reg, ancilla)
    circuit.x(data_reg[1])
    return circuit


def main():
        print("(c)")
        data =  [5,2,4,5,3,7,6,1]
        address = QuantumRegister(4)
        data = QuantumRegister(2)
        c = ClassicalRegister(0)
        qc = QuantumCircuit(address,data,c) 
        qc.h([0,1,2])
        qc.append(qram(data), range(6))
        
        qc.append(oracle_c(2), range(3))
        qc.measure_all()
        backend = Aer.get_backend('qasm_simulator')
        job = execute(qc, backend)
        result = job.result()
        count =result.get_counts()
        print(count)
        print(qc)    
        
        

if __name__ == "__main__":
    main()

(c)
{'110110 ': 120, '001010 ': 118, '111001 ': 117, '010100 ': 141, '100111 ': 125, '110000 ': 158, '110101 ': 119, '011011 ': 126}
         ┌───┐┌───────┐┌─────────┐ ░ ┌─┐               
q2669_0: ┤ H ├┤0      ├┤0        ├─░─┤M├───────────────
         ├───┤│       ││         │ ░ └╥┘┌─┐            
q2669_1: ┤ H ├┤1      ├┤1 Oracle ├─░──╫─┤M├────────────
         ├───┤│       ││         │ ░  ║ └╥┘┌─┐         
q2669_2: ┤ H ├┤2      ├┤2        ├─░──╫──╫─┤M├─────────
         └───┘│  QRAM │└─────────┘ ░  ║  ║ └╥┘┌─┐      
q2669_3: ─────┤3      ├────────────░──╫──╫──╫─┤M├──────
              │       │            ░  ║  ║  ║ └╥┘┌─┐   
q2670_0: ─────┤4      ├────────────░──╫──╫──╫──╫─┤M├───
              │       │            ░  ║  ║  ║  ║ └╥┘┌─┐
q2670_1: ─────┤5      ├────────────░──╫──╫──╫──╫──╫─┤M├
              └───────┘            ░  ║  ║  ║  ║  ║ └╥┘
 meas: 6/═════════════════════════════╩══╩══╩══╩══╩══╩═
                                      0  1  2  3  4  5 
